In [1]:
import pandas as pd
from openai import OpenAI
import numpy as np
import time


client = OpenAI()


In [2]:
# Set option to display more columns
pd.set_option('display.max_columns', None)  # None means unlimited
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_colwidth', None)


In [2]:
all_echo = pd.read_csv("20240417_all_60days_closest.csv")

In [3]:
len(all_echo)

2560

In [4]:
all_echo.columns

Index(['note_id_x', 'subject_id', 'hadm_id', 'note_type', 'note_seq_x',
       'charttime', 'storetime', 'text', 'study_id', 'study_datetime',
       'note_id_y', 'note_seq_y', 'note_charttime', 'time_difference'],
      dtype='object')

In [5]:
note_ids = [
    '12211141-DS-20',
    '12466651-DS-20',
    '16086976-DS-10',
    '10966093-DS-22',
    '11994782-DS-2',
    '13860987-DS-16',
    '17055745-DS-4',
    '18001424-DS-16',
    '18436961-DS-24',
    '19275910-DS-7',
    '17121541-DS-18',
    '17618022-DS-25',
    '10239721-DS-13',
    '12137011-DS-32',
    '12323804-DS-16',
    '16240920-DS-16',
    '16009371-DS-19',
    '19242994-DS-19',
    '10367718-DS-25',
    '12660752-DS-11',
    '13297743-DS-111',
    '13707062-DS-17',
    '14290075-DS-30',
    '15461582-DS-27',
    '16169603-DS-19',
    '17673024-DS-7',
    '17897160-DS-10',
    '19066200-DS-17',
    '19408205-DS-18',
    '11622732-DS-3',
    '10629866-DS-12',
    '11006886-DS-20'
]

In [6]:
tricky_ones = all_echo[all_echo['note_id_x'].isin(note_ids)]
tricky_ones.reset_index(drop=True, inplace=True)

In [7]:
def append_columns(df):
    # Define all possible column labels from the sample outputs
    column_labels = [
        "Ejection Fraction (EF)", "tokens used"
    ]
    
    # Append missing columns to echo_df
    for column in column_labels:
        if column not in df.columns:
            df[column] = None  # Initialize new columns with None values
    
    return df

In [8]:
def GPT_extract_data_prev(i, df):
    completion = client.chat.completions.create(
      model="gpt-4-turbo",
      # model="gpt-3.5-turbo",
      messages=[
        {"role": "system", 
                "content":
                    "You are an experienced Machine Learning Engineer, Data Scientist and an experienced Cardiologist. \n"
                    "Your goal is to extract target data (Left Ventricle Ejection Fraction or EF) from the given cardiology note. \n"
                    "The study of interest is only TTE. EF values from TEE, Nuclear, Cardiac MRC, etc should not be considered. \n"
                    "Your output should contain nothing other than the data. \n"
                    "Your output should be an integer. If no such data available, put 'NAN' to the value. \n"
                    "Your output should contain nothing other than the data itself.\n"
                    "You can add parentheses after the data for a very brief explanation if you are not certain.\n"
                    "For example, if the EF mentioned in the note is a range, you should take an average of that range, then put the range in the parentheses.\n"
                    "Sample output: 60 (55-65) \n"
                    "For another example, if there are different TTE studies and you are not sure which EF to use, do a best guess and specify that situation in the parentheses.\n"
                    "Sample output: 55 (multiple studies, took the value from the 1st one) \n"
                    "Be very careful not to take the EF values from the patient's medical history."
            },
        {"role": "user", "content": df.iloc[i,7]}
      ]
    )
    return completion



In [9]:
def GPT_extract_data(i, df):
    completion = client.chat.completions.create(
      model="gpt-4-turbo",
      # model="gpt-3.5-turbo",
      messages=[
        {"role": "system", 
                "content":
                    "You are an experienced Data Scientist and Cardiologist. \n"
                    "Your goal is to extract the Left Ventricle Ejection Fraction (also known as LVEF or EF) from doctors' notes about a patient during a hospital stay. \n"
                    "The given note will contain a lot of irrelevant information that is not the EF, which should be ignored. \n"
                    "You should only report EF values from a transthoracic echocardiogram study, also known as an Echo or TTE. \n"
                    "TTE results will often (but not always) appear under a heading like 'Pertinent Results', 'Pertinent Studies', or 'Imaging Reports'. \n"
                    "EF values from transesophageal, TEE, Nuclear, MRI, cardiac perfusion, x-ray, ct, PUMP, bedside ultrasound, or any other non-TTE study types should not be considered. \n"
                    "Do not report any EF values recorded as part of the patient's medical history. Only record values from a TTE study that took place during this hospital stay. \n"
                    "Notes will often have sections titled 'Brief Hospital Course' and 'History of Present Illness'. EF values reported early in these sections where it talks about patient history or what the patient presented with should be ignored. \n"
                    "The EF in the note may be given as an integer value (e.g. 55%), as a range (e.g. 55-60%), or as an inequality (e.g. >55%). \n"
                    "Ignore qualitative notes about the EF such as 'low normal', 'recovered', 'severely depressed', 'elevated', etc. Only look at numerical values. \n"
                    "Output format: \n"
                    "1. Your output should only contain the desired data, and nothing other than that.\n"
                    "2. If there is only one TTE/Echo study in the note with a valid EF value, report just the integer value, the range or the inequality. \n"
                    "   Sample outputs: '55'; '55-60'; '>55'.\n"
                    "3. If there is no valid EF data in the note or the only EF values reported are from medical history, just report 'NAN'. \n"
                    "4. If two or more TTE/Echo studies(not from medical history) were performed on the patient, report 'Multiple TTE/Echo Studies (x/y):' followed by EF values separates by commas. \n"
                    "   x >= 1 indicates the number of TTE/Echo studies that give a EF value, y >= 2 indicates how many TTE/Echo studies are there.\n"
                    "   You should output in this format only when there are two or more TTE/Echo studies and have one or more EF values given in the note. \n"
                    "   If there is only one TTE/Echo study, you should follow one of the output formats as specified in (2), (3) or (5). \n"
                    "   If there are multiple studies but no specific EF value given, report 'NAN'.\n"
                    "   Sample outputs: 'Multiple TTE/Echo Studies(1/2): 55, _'; 'Multiple TTE/Echo Studies(2/3): 25, _, 55'; 'Multiple TTE/Echo Studies(2/2): 40, 55'. \n"
                    "   This does not apply if the same EF is just reported multiple times; for example, if there's only one TTE/Echo study and it is listed in one place as '20%' and in another as 'EF 20%', just report '20'. \n"
                    "   Some EF values are redacted and reported something like 'LVEF = ___ %'. Ignore these redacted values, but still report if there are multiple different TTTE/EchoTE studies even if some have redacted EF values. \n"
                    "5. Some notes may not be clear and can be confusing. If you are uncertain about the correct answer, you should report: 'Uncertain(Reason/Explanation) ' followed by your best guess.\n"
                    "   Sample outputs: 'Uncertain(Not sure if data is from history): 55'; 'Uncertain(Not sure if there's multiple TTE/Echo studies): 66'.\n"
            },
        {"role": "user", "content": df.iloc[i,7]}
      ]
    )
    return completion




In [29]:
def NLP_notes(df_ori, output_name):
    num_rows = df_ori.shape[0]
    df = df_ori.copy()
    
    for i in range(num_rows):    
        completion = GPT_extract_data(i, df_ori)  # Assuming this function returns the completion object
        total_tokens = completion.usage.total_tokens  # Extracting the total_tokens value
        update_row_from_completion(df, i, completion, total_tokens)
        if i % 10 == 0:
            print(i)
        if i % 100 == 0:
            df.to_csv("temp_saving.csv", index=False) 

    processed_filename = f"{output_name}_processed.csv"
    note_dropped_filename = f"{output_name}_processed_note_dropped.csv"
    
    df.to_csv(processed_filename, index=False)
    print(f"Saved processed file as: {processed_filename}")
    
    df_drop_note = df.copy()
    df_drop_note = df_drop_note.drop(columns=['text'])
    df_drop_note.to_csv(note_dropped_filename, index=False)
    print(f"Saved note-dropped file as: {note_dropped_filename}")
    return df_drop_note


In [ ]:
def update_row_from_completion(df, i, completion, total_tokens):
    completion_text = completion.choices[0].message.content  # Extracting the text part of the completion message
    lines = completion_text.split('\n')
    for line in lines:
        if ': ' in line:  # Ensure the line contains a key-value pair
            key, value = line.split(': ', 1)
            value = value.strip()
            # Replace 'NAN' text with np.nan
            if value == 'NAN':
                value = np.nan
            # Check if the key exists as a column in the DataFrame; if it does, update the value
            if key in df.columns:
                df.at[i, key] = value
            else:
                print(f"Column {key} does not exist in the DataFrame.")
                
    df.at[i, 'tokens used'] = total_tokens


In [22]:
print(completion.choices[0].message.content)
print(completion.usage.total_tokens)

87.5 (75-100)
6107


In [8]:
all_echo = append_columns(all_echo)

In [11]:
output_name = '20240424_GPT4_turbo_EF_only_v2'
df_temp = all_echo
num_rows = df_temp.shape[0]
df = df_temp.copy()

for i in range(num_rows):    
    completion = GPT_extract_data(i, df_temp)  # Assuming this function returns the completion object
    completion_text = completion.choices[0].message.content
    total_tokens = completion.usage.total_tokens  # Extracting the total_tokens value
    df.at[i, 'Ejection Fraction (EF)'] = completion_text
    df.at[i, 'tokens used'] = total_tokens
    if i % 10 == 0:
        print(i)
    if i % 100 == 0:
       df.to_csv("temp_saving.csv", index=False) 

processed_filename = f"{output_name}_processed.csv"
note_dropped_filename = f"{output_name}_processed_note_dropped.csv"

df.to_csv(processed_filename, index=False)
print(f"Saved processed file as: {processed_filename}")

df_drop_note = df.copy()
df_drop_note = df_drop_note.drop(columns=['text'])
df_drop_note.to_csv(note_dropped_filename, index=False)
print(f"Saved note-dropped file as: {note_dropped_filename}")


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [10]:
tricky_ones = append_columns(tricky_ones)
output_name = '20240420_GPT4_turbo_tricky_EF_only_v2.3'
df_temp = tricky_ones
num_rows = df_temp.shape[0]
df = df_temp.copy()

for i in range(num_rows):    
    completion = GPT_extract_data(i, df_temp)  # Assuming this function returns the completion object
    completion_text = completion.choices[0].message.content
    total_tokens = completion.usage.total_tokens  # Extracting the total_tokens value
    df.at[i, 'Ejection Fraction (EF)'] = completion_text
    df.at[i, 'tokens used'] = total_tokens
    if i % 10 == 0:
        print(i)
    if i % 100 == 0:
       df.to_csv("temp_saving.csv", index=False) 

# processed_filename = f"{output_name}_processed.csv"
# note_dropped_filename = f"{output_name}_processed_note_dropped.csv"

# df.to_csv(processed_filename, index=False)
# print(f"Saved processed file as: {processed_filename}")

# df_drop_note = df.copy()
# df_drop_note = df_drop_note.drop(columns=['text'])
# df_drop_note.to_csv(note_dropped_filename, index=False)
print(f"Saved note-dropped file as: {note_dropped_filename}")


C:\Users\gzy\AppData\Local\Temp\ipykernel_33928\2271959150.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = None  # Initialize new columns with None values
C:\Users\gzy\AppData\Local\Temp\ipykernel_33928\2271959150.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = None  # Initialize new columns with None values


0
10
20
30


NameError: name 'note_dropped_filename' is not defined

In [11]:
assistant = client.beta.assistants.create(
  name="Cardiologist",
  instructions=                    
    "You are an experienced Cardiologist. \n"
    "Your goal is to extract target data (Left Ventricle Ejection Fraction or EF) from the given cardiology note. \n"
    "The study of interest is only TTE. EF values from TEE, Nuclear, Cardiac MRC, etc should not be considered. \n"
    "Your output should contain nothing other than the data. \n"
    "Your output should be an integer. If no such data available, put 'NAN' to the value. \n"
    "Your output should contain nothing other than the data itself.\n"
    "You can add parentheses after the data for a very brief explanation if you are not certain.\n"
    "For example, if the EF mentioned in the note is a range, you should take an average of that range, then put the range in the parentheses.\n"
    "Sample output: 60 (55-65) \n"
    "For another example, if there are different TTE studies and you are not sure which EF to use, do a best guess and specify that situation in the parentheses.\n"
    "Sample output: 55 (multiple studies, took the value from the 1st one) \n"
    "Be very careful not to take the EF values from the patient's medical history.",
  model="gpt-4-turbo",
)

In [8]:
thread = client.beta.threads.create()

In [9]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=tricky_ones.iloc[0,7]
)

In [12]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)


In [ ]:
while True:
    if run.status == 'completed': 
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        print(messages)
        break
    else:
        print(run.status)
        time.sleep(1)